#Import Dataframes and Instantiate Cerebro Backtrader Engine

In [106]:
# import pandas as pd
import backtrader as bt
from backtrader import cerebro
import backtrader.feeds as btfeeds

os.getcwd()
os.chdir(r'C:\Users\angel\Documents\Documents\GitHub\Group-Project-Hill-Wiley-Gwinn\Data')

# df_stock = pd.read_csv(r'C:\Users\angel\Documents\Documents\GitHub\Group-Project-Hill-Wiley-Gwinn\Data\AAPL.csv')
# df_fin = pd.read_csv(r"C:\Users\angel\Documents\Documents\GitHub\Group-Project-Hill-Wiley-Gwinn\Data\AAPLfinancials.csv")

cerebro = bt.Cerebro()

pricedf = bt.feeds.YahooFinanceCSVData(dataname='AAPL.csv', reverse=False)
ratiodf = bt.feeds.YahooFinanceCSVData(dataname='AAPLfinancials.csv', reverse=False)

cerebro.adddata(pricedf, ratiodf)
print(pricedf, ratiodf) #why are the df's not printing?

cerebro.broker.set_cash(2500)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

<backtrader.feeds.yahoo.YahooFinanceCSVData object at 0x000002C58FA8FF70> <backtrader.feeds.yahoo.YahooFinanceCSVData object at 0x000002C58FB342B0>
Starting Portfolio Value: 2500.00
Final Portfolio Value: 2500.00


#Create MACD Strategy Class

In [7]:
import backtrader.indicators as btind
import backtrader as bt
import datetime

class MACD_Crossover(bt.Strategy):
    """
    This class serves as template for developing trading strategies using the MACD
    technical indicator. The buy logic is dependant on a crossover that occurs with
    the MACD indicator.
    """

    params = (
        # Standard MACD Parameters
        ('macd1', 12),
        ('macd2', 26),
        ('macdsig', 9),
        ('atrperiod', 14),  # ATR Period (standard)
        ('atrdist', 3.0),   # ATR distance for stop price
        ('smaperiod', 30),  # SMA Period (pretty standard)
        ('dirperiod', 10),  # Lookback period to consider SMA trend direction
    )

    def log(self, txt, dt=None):
        '''Logging function for strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.macd = bt.indicators.MACD(self.data,
                                        period_me1=self.p.macd1,
                                        period_me2=self.p.macd2,
                                        period_signal=self.p.macdsig)

        # self.order = None
        # self.buyprice = None
        # self.buycomm = None

        # Cross of macd.macd and macd.signal
        self.mcross = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)

        # set stop price indicator
        self.atr = bt.indicators.ATR(self.data, period=self.p.atrperiod)

        self.sma = bt.indicators.SMA(self.data, period=self.p.smaperiod)
        self.smadir = self.sma - self.sma(-self.p.dirperiod)

    def notify_order(self, order):
        
        # if order.status in [order.Submitted, order.Accepted]:
        #     # Buy/Sell order submitted/accepted to/by broker
        #     return
        
        # # Check if an order has been completed
        # # Attention: broker could reject order if not enough cash
        # if order.status in [order.Completed]:
        #     if order.isbuy():
        #         self.log('BUY EXECUTED, %.2f' % order.executed.price)
        #     elif order.issell():
        #         self.log('SELL EXECUTED, %.2f' % order.executed.price)

        #     self.bar_executed = len(self)

        # elif order.status in [order.Canceled, order.Margin, order.Rejected]:
        #     self.log('Order Canceled/Margin/Rejected')

        # # Write down: no pending order
        # self.order = None

    def next(self):
        if self.CrossOver == 1:
            # buy stock
            pass
        elif self.CrossOver == -1:
            # sell stock
            pass
        # else:
            pass  

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.data.close[0])

        # Check if an order is pending - will need to remove for when we trade multiple stocks
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            if self.mcross == 1:
                self.order = self.buy()
                self.log('BUY CREATE, %.2f' % self.data.close[0])
            
        else: # Already in a position
            if self.mcross == -1:
                self.order = self.sell()
                self.log('SELL CREATE, %.2f' % self.data.close[0])

if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(MACD_Crossover)

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    # modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    # datapath = os.path.join(modpath, r'C:\Users\angel\Documents\Documents\GitHub\Group-Project-Hill-Wiley-Gwinn\Data')
    os.chdir(r'C:/Users/angel/Documents/Documents/GitHub/Group-Project-Hill-Wiley-Gwinn/Data')
    print(os.listdir())
    # Create a Data Feed
    pricedata = bt.feeds.YahooFinanceCSVData(
        dataname='AAPL.csv',
        # Do not pass values before this date
        # fromdate=datetime.datetime(2000, 1, 1),
        # # Do not pass values before this date
        # todate=datetime.datetime(2000, 12, 31),
        # # Do not pass values after this date
        reverse=False) 

    # Add the Data Feed to Cerebro
    cerebro.adddata(pricedata)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
 
    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # os.getcwd()
    # os.chdir(r'C:\Users\angel\Documents\Documents\GitHub\Group-Project-Hill-Wiley-Gwinn\Data')

    # # df_stock = pd.read_csv(r'C:\Users\angel\Documents\Documents\GitHub\Group-Project-Hill-Wiley-Gwinn\Data\AAPL.csv')
    # # df_fin = pd.read_csv(r"C:\Users\angel\Documents\Documents\GitHub\Group-Project-Hill-Wiley-Gwinn\Data\AAPLfinancials.csv")

    # cerebro = bt.Cerebro()

    # pricedf = bt.feeds.YahooFinanceCSVData(dataname='AAPL.csv', reverse=False)
    # ratiodf = bt.feeds.YahooFinanceCSVData(dataname='AAPLfinancials.csv', reverse=False)

    # cerebro.adddata(pricedf, ratiodf)
    # print(pricedf, ratiodf) #why are the df's not printing?

    # cerebro.broker.set_cash(2500)
    # print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    # print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())



# macd1 = MACD_Crossover()

        
    


['AAPL.csv', 'AAPLfinancials.csv']
Starting Portfolio Value: 100000.00
2016-12-21, Close, 27.59


AttributeError: 'Lines_LineSeries_LineIterator_DataAccessor_Strateg' object has no attribute 'order'

#Create SMA Crossover 1 Strategy Class

#Create EMA Crossover 2 Strategy Class

In [ ]:
class SMA_EMA_Crossover(bt.Strategy):

    def __init__(self):

        sma1 = btind.SimpleMovingAverage(self.data)
        ema1 = btind.ExponentialMovingAverage()

        close_over_sma = self.data.close > sma1
        close_over_ema = self.data.close > ema1
        sma_ema_diff = sma1 - ema1

        buy_sig = bt.And(close_over_sma, close_over_ema, sma_ema_diff > 0)

    def next(self):

        if buy_sig:
            self.buy()

#Create Mean Reversion Strategy

#Buy and hold baseline strategy